# Akuisisi senyawa aktif

In [4]:
import pubchempy as pcp
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import xmltodict
import os
import ssl
import certifi
import urllib3

ssl._create_default_https_context = ssl._create_unverified_context

#Matikan warning SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#Verifikasi PubChem

In [23]:
import requests
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_cids(compound_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/cids/TXT"
    r = requests.get(url, timeout=20, verify=False)
    if r.status_code == 200 and r.text.strip():
        return r.text.strip().split("\n")
    return []

def get_properties(cid):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularFormula,MolecularWeight,CanonicalSMILES,IsomericSMILES,SMILES,IUPACName/JSON"
    r = requests.get(url, timeout=20, verify=False)
    if r.status_code == 200:
        data = r.json().get("PropertyTable", {}).get("Properties", [{}])[0]
        props = {}
        props["CID"] = cid
        props["Formula"] = data.get("MolecularFormula", "-")
        props["MolecularWeight"] = data.get("MolecularWeight", "-")
        props["IUPACName"] = data.get("IUPACName", "-")

        if "CanonicalSMILES" in data and data["CanonicalSMILES"]:
            props["SMILES"] = data["CanonicalSMILES"]
        elif "SMILES" in data and data["SMILES"]:
            props["SMILES"] = data["SMILES"]
        elif "IsomericSMILES" in data and data["IsomericSMILES"]:
            props["SMILES"] = data["IsomericSMILES"]
        else:
            props["SMILES"] = "-"
        return props
    return None

def search_compounds(compounds):
    results = []
    not_found = []

    for name in compounds:
        print("="*70)
        print(f"Senyawa : {name}")
        cids = get_cids(name)

        if not cids:
            print("❌ Tidak ditemukan di PubChem")
            results.append({
                "Compound": name,
                "CID": "",
                "Formula": "",
                "MolecularWeight": "",
                "IUPACName": "",
                "SMILES": "",
                "Status": "❌ Tidak ditemukan"
            })
            not_found.append(name)
            continue

        print(f"✅ Ditemukan {len(cids)} CID: {', '.join(cids)}")
        for cid in cids:
            props = get_properties(cid)
            if props:
                print(f"   CID {props['CID']}")
                print(f"      Formula     : {props['Formula']}")
                print(f"      MolWeight   : {props['MolecularWeight']}")
                print(f"      IUPAC Name  : {props['IUPACName']}")
                print(f"      SMILES      : {props['SMILES']}")
                results.append({
                    "Compound": name,
                    "CID": props["CID"],
                    "Formula": props["Formula"],
                    "MolecularWeight": props["MolecularWeight"],
                    "IUPACName": props["IUPACName"],
                    "SMILES": props["SMILES"],
                    "Status": "✅ Ditemukan"
                })
            else:
                print(f"   CID {cid} → ❌ Data tidak tersedia")
                results.append({
                    "Compound": name,
                    "CID": cid,
                    "Formula": "",
                    "MolecularWeight": "",
                    "IUPACName": "",
                    "SMILES": "",
                    "Status": "❌ Data tidak tersedia"
                })

    # Simpan semua hasil ke Excel
    df = pd.DataFrame(results)
    df.to_excel("D:/semhas/Data/senyawa_pubchem.xlsx", index=False)
    print("\n✅ Semua data tersimpan di 'senyawa_pubchem.xlsx'")

    if not_found:
        print("\nDaftar senyawa yang tidak ditemukan:")
        for nf in not_found:
            print(f" - {nf}")

# ==== CONTOH PENGGUNAAN ====
compounds = [
    "4-aminobenzoic acid",
    "Citric acid",
    "L-phenylalanine",
    "Trans-3-indoleacrylic acid",
    "DL-tryptophan",
    "4-O-feruloyl-D-quinic acid",
    "Murrangatin",
    "Alpha-lapachone",
    "Hainanmurpanin",
    "Murraol",
    "Murralongin",
    "(1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one",
    "Paniculatin",
    "Limonene, (+/-)-",  
    "Pheophorbide a",
    "SB236057"
]

search_compounds(compounds)


Senyawa : 4-aminobenzoic acid
✅ Ditemukan 1 CID: 978
   CID 978
      Formula     : C7H7NO2
      MolWeight   : 137.14
      IUPAC Name  : 4-aminobenzoic acid
      SMILES      : C1=CC(=CC=C1C(=O)O)N
Senyawa : Citric acid
✅ Ditemukan 1 CID: 311
   CID 311
      Formula     : C6H8O7
      MolWeight   : 192.12
      IUPAC Name  : 2-hydroxypropane-1,2,3-tricarboxylic acid
      SMILES      : C(C(=O)O)C(CC(=O)O)(C(=O)O)O
Senyawa : L-phenylalanine
✅ Ditemukan 1 CID: 6140
   CID 6140
      Formula     : C9H11NO2
      MolWeight   : 165.19
      IUPAC Name  : (2S)-2-amino-3-phenylpropanoic acid
      SMILES      : C1=CC=C(C=C1)C[C@@H](C(=O)O)N
Senyawa : Trans-3-indoleacrylic acid
✅ Ditemukan 1 CID: 5375048
   CID 5375048
      Formula     : C11H9NO2
      MolWeight   : 187.19
      IUPAC Name  : (E)-3-(1H-indol-3-yl)prop-2-enoic acid
      SMILES      : C1=CC=C2C(=C1)C(=CN2)/C=C/C(=O)O
Senyawa : DL-tryptophan
✅ Ditemukan 1 CID: 1148
   CID 1148
      Formula     : C11H12N2O2
      MolWeight  

#smiles Limonene, (+/-)-(CID=22311)

In [24]:
def get_properties_by_cid(cid):
    """Ambil Formula, MolWeight, IUPAC Name, dan SMILES dari CID PubChem"""
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularFormula,MolecularWeight,CanonicalSMILES,IsomericSMILES,SMILES,IUPACName/JSON"
    r = requests.get(url, timeout=20)
    
    if r.status_code == 200:
        data = r.json().get("PropertyTable", {}).get("Properties", [{}])[0]
        return {
            "CID": cid,
            "Formula": data.get("MolecularFormula", "-"),
            "MolWeight": data.get("MolecularWeight", "-"),
            "IUPACName": data.get("IUPACName", "-"),
            "SMILES": data.get("CanonicalSMILES") or data.get("IsomericSMILES") or data.get("SMILES", "-")
        }
    return None

# Ambil data Limonene
cid = "22311"
props = get_properties_by_cid(cid)

# Tampilkan hasil
print("="*60)
print(f"CID         : {props['CID']}")
print(f"Formula     : {props['Formula']}")
print(f"MolWeight   : {props['MolWeight']}")
print(f"IUPAC Name  : {props['IUPACName']}")
print(f"SMILES      : {props['SMILES']}")
print("="*60)

# Buat DataFrame untuk disimpan
df_limonene = pd.DataFrame([props])

# Simpan ke Excel, sheet baru "Limonene"
excel_file = "D:/semhas/Data/senyawa_pubchem.xlsx"
with pd.ExcelWriter(excel_file, mode="a", engine="openpyxl") as writer:
    df_limonene.to_excel(writer, sheet_name="Limonene", index=False)

print(f"✅ Data Limonene tersimpan di sheet 'Limonene' pada {excel_file}")


CID         : 22311
Formula     : C10H16
MolWeight   : 136.23
IUPAC Name  : 1-methyl-4-prop-1-en-2-ylcyclohexene
SMILES      : CC1=CCC(CC1)C(=C)C
✅ Data Limonene tersimpan di sheet 'Limonene' pada D:/semhas/Data/senyawa_pubchem.xlsx


# (1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one (CACTUS)

In [19]:
import urllib.parse

def get_from_cactus(compound_name):
    base_url = "https://cactus.nci.nih.gov/chemical/structure"
    formats = {
        "SMILES": "smiles",
        "Formula": "formula",
        "MolecularWeight": "mw",
        "IUPAC_Name": "iupac_name"
    }

    results = {"Compound": compound_name}
    encoded_name = urllib.parse.quote(compound_name)

    for key, fmt in formats.items():
        url = f"{base_url}/{encoded_name}/{fmt}"
        try:
            r = requests.get(url, timeout=20)
            if r.status_code == 200 and r.text.strip():
                results[key] = r.text.strip()
            else:
                results[key] = "-"
        except Exception as e:
            results[key] = f"Error: {e}"
    return results


# ==== Contoh penggunaan ====
compound_name = "(1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one"

props = get_from_cactus(compound_name)

print("="*70)
print(f"Nama Senyawa   : {props['Compound']}")
print(f"SMILES         : {props['SMILES']}")
print(f"Formula        : {props['Formula']}")
print(f"MolecularWeight: {props['MolecularWeight']}")
print(f"IUPAC Name     : {props['IUPAC_Name']}")
print("="*70)


Nama Senyawa   : (1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one
SMILES         : Clc1ccc(cc1)\C=C\C2=CC=C3[C@@H]4C[C@@H](CN(C4)C(=O)c5cncnc5)CN3C2=O
Formula        : C24H21ClN4O2
MolecularWeight: 432.9085
IUPAC Name     : -


#Mencari cid dari smiles (1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one 
#di pubchem

In [26]:
def get_cid_from_smiles(smiles):
    """Cari CID PubChem berdasarkan SMILES"""
    encoded = urllib.parse.quote(smiles)
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{encoded}/cids/TXT"
    r = requests.get(url, timeout=20)
    if r.status_code == 200 and r.text.strip():
        return r.text.strip().split("\n")  # bisa lebih dari satu CID
    return []

def get_properties_by_cid(cid):
    """Ambil Formula, MolWeight, IUPAC Name, dan SMILES dari CID PubChem"""
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularFormula,MolecularWeight,CanonicalSMILES,IsomericSMILES,SMILES,IUPACName/JSON"
    r = requests.get(url, timeout=20)
    
    if r.status_code == 200:
        data = r.json().get("PropertyTable", {}).get("Properties", [{}])[0]
        return {
            "CID": cid,
            "Formula": data.get("MolecularFormula", "-"),
            "MolWeight": data.get("MolecularWeight", "-"),
            "IUPACName": data.get("IUPACName", "-"),
            "SMILES": data.get("CanonicalSMILES") or data.get("IsomericSMILES") or data.get("SMILES", "-")
        }
    return None

def search_smiles_and_save(smiles, excel_file="D:/semhas/Data/senyawa_pubchem.xlsx", sheet_name="From_SMILES"):
    print("="*70)
    print(f"SMILES yang dicari: {smiles}")
    
    cids = get_cid_from_smiles(smiles)
    results = []

    if not cids:
        print("❌ CID tidak ditemukan di PubChem")
        return

    print(f"✅ Ditemukan CID(s): {', '.join(cids)}")
    for cid in cids:
        props = get_properties_by_cid(cid)
        if props:
            print(f"--- CID {props['CID']} ---")
            print(f"Formula       : {props['Formula']}")
            print(f"MolWeight     : {props['MolWeight']}")
            print(f"IUPAC Name    : {props['IUPACName']}")
            print(f"SMILES        : {props['SMILES']}")
            results.append(props)
        else:
            print(f"CID {cid} → ❌ Data tidak tersedia")

    # Simpan ke Excel di sheet baru
    if results:
        df = pd.DataFrame(results)
        with pd.ExcelWriter(excel_file, mode="a", engine="openpyxl") as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"\n✅ Data disimpan di sheet '{sheet_name}' pada {excel_file}")

# SMILES
smiles = r"Clc1ccc(cc1)\C=C\C2=CC=C3[C@@H]4C[C@@H](CN(C4)C(=O)c5cncnc5)CN3C2=O" 
search_smiles_and_save(smiles)


SMILES yang dicari: Clc1ccc(cc1)\C=C\C2=CC=C3[C@@H]4C[C@@H](CN(C4)C(=O)c5cncnc5)CN3C2=O
✅ Ditemukan CID(s): 11886484
--- CID 11886484 ---
Formula       : C24H21ClN4O2
MolWeight     : 432.9
IUPAC Name    : (1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one
SMILES        : C1[C@H]2CN(C[C@@H]1C3=CC=C(C(=O)N3C2)/C=C/C4=CC=C(C=C4)Cl)C(=O)C5=CN=CN=C5

✅ Data disimpan di sheet 'From_SMILES' pada D:/semhas/Data/senyawa_pubchem.xlsx


In [ ]:
#recap

In [ ]:
!pip install tabulate

In [32]:
import pandas as pd
import requests
import urllib.parse
from tabulate import tabulate

# ----- Fungsi bantu -----
def get_cids_from_name(name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{urllib.parse.quote(name)}/cids/TXT"
    r = requests.get(url, timeout=20)
    if r.status_code == 200 and r.text.strip():
        return r.text.strip().split("\n")
    return []

def get_properties_by_cid(cid):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularFormula,MolecularWeight,CanonicalSMILES,IsomericSMILES,SMILES,IUPACName/JSON"
    r = requests.get(url, timeout=20)
    if r.status_code == 200:
        data = r.json().get("PropertyTable", {}).get("Properties", [{}])[0]
        return data.get("CanonicalSMILES") or data.get("IsomericSMILES") or data.get("SMILES", "-")
    return "-"

# ----- Keterangan manual -----
manual_notes = {
    "(1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one":
        "Ditentukan manual via CACTUS → Terverifikasi PubChem",
    "Limonene, (+/-)-":
        "Hanya tersedia rasemat (Xu et al., 2017) → Terverifikasi PubChem"
}

# ----- Daftar senyawa -----
compounds = [
    "4-aminobenzoic acid",
    "Citric acid",
    "L-phenylalanine",
    "Trans-3-indoleacrylic acid",
    "DL-tryptophan",
    "4-O-feruloyl-D-quinic acid",
    "Murrangatin",
    "Alpha-lapachone",
    "Hainanmurpanin",
    "Murraol",
    "Murralongin",
    "(1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one",
    "Paniculatin",
    "Limonene, (+/-)-",
    "Pheophorbide a",
    "SB236057"
]

# ----- Proses semua senyawa -----
results = []
no = 1
for name in compounds:
    cids = get_cids_from_name(name)
    if cids:
        for cid in cids:
            smiles = get_properties_by_cid(cid)
            keterangan = manual_notes.get(name, "Diverifikasi")
            results.append({
                "No.": no,
                "Compound": name,
                "CID": cid,
                "SMILES": smiles,
                "Status Di PubChem": "Ada",
                "Keterangan": keterangan
            })
            no += 1
    else:
        keterangan = manual_notes.get(name, "Belum Diverifikasi")
        results.append({
            "No.": no,
            "Compound": name,
            "CID": "-",
            "SMILES": "-",
            "Status Di PubChem": "Tidak Ada",
            "Keterangan": keterangan
        })
        no += 1

# ----- Buat DataFrame -----
df = pd.DataFrame(results)
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

# ----- Simpan ke Excel (sheet berbeda) -----
output_file = "D:/semhas/Data/senyawa_pubchem.xlsx"
sheet_name = "Hasil_Senyawa"  # bisa diganti tiap run, misalnya "Batch1", "Batch2"

with pd.ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ Data tersimpan di file '{output_file}' pada sheet '{sheet_name}'")


+-------+---------------------------------------------------------------------------------------------------------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------+---------------------+------------------------------------------------------------------+
|   No. | Compound                                                                                                                  | CID      | SMILES                                                                                                                             | Status Di PubChem   | Keterangan                                                       |
+=======+===========================================================================================================================+==========+==========================================================================================================================

# SwissAdme

In [7]:
import pandas as pd

# File Excel
compound_file = 'D:/semhas/Data/senyawa_pubchem.xlsx'
swissadme_file = 'D:/semhas/Data/swissadme_hasil.csv'

# Baca sheet verifikasi_senyawa
df_verif = pd.read_excel(compound_file, sheet_name='Hasil_Senyawa')
print(f"Jumlah baris verifikasi_senyawa: {len(df_verif)}")

# Baca file SwissADME
df_swiss = pd.read_csv(swissadme_file)
print(f"Jumlah baris SwissADME: {len(df_swiss)}")

# Pastikan jumlah baris sama
if len(df_verif) != len(df_swiss):
    raise ValueError("Jumlah baris SwissADME dan verifikasi_senyawa tidak sama!")

# Tambahkan kolom Compound dan CID dari verifikasi_senyawa
df_swiss['Compound'] = df_verif['Compound']
df_swiss['CID'] = df_verif['CID']

# Tambahkan Status akhir berdasarkan Bioavailability Score
df_swiss['Status akhir'] = df_swiss['Bioavailability Score'].apply(
    lambda x: 'Dipilih' if x >= 0.55 else 'Tidak dipilih'
)

# Tambahkan kolom No.
df_swiss.insert(0, 'No.', range(1, len(df_swiss) + 1))

# Simpan ke sheet baru di file Excel yang sama
with pd.ExcelWriter(compound_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_swiss.to_excel(writer, sheet_name='swissadme_senyawa', index=False)

print(f"\n✅ Data SwissADME berhasil disimpan di sheet 'swissadme_senyawa' dalam file {compound_file}")

# --- Tambahan: Hitung jumlah senyawa unik dan jumlah total SMILES ---
jumlah_senyawa = df_verif['Compound'].nunique()
jumlah_smiles = len(df_verif)

print("\n📊 Ringkasan Data Senyawa:")
print(f"Jumlah senyawa unik   : {jumlah_senyawa} senyawa")
print(f"Jumlah total SMILES   : {jumlah_smiles} SMILES")

# Tampilkan daftar senyawa unik
print("\nDaftar senyawa unik:")
for i, senyawa in enumerate(df_verif['Compound'].unique(), start=1):
    print(f"{i}. {senyawa}")

# --- Tambahan: Ringkasan berdasarkan Status akhir ---
dipilih = df_swiss[df_swiss['Status akhir'] == 'Dipilih']
tidak_dipilih = df_swiss[df_swiss['Status akhir'] == 'Tidak dipilih']

print("\n📌 Ringkasan Status Akhir:")
print(f"Senyawa dengan status 'Dipilih'     : {dipilih['Compound'].nunique()} senyawa ({len(dipilih)} SMILES)")
print(f"Senyawa dengan status 'Tidak dipilih': {tidak_dipilih['Compound'].nunique()} senyawa ({len(tidak_dipilih)} SMILES)")

# Tampilkan daftar senyawa dipilih & tidak dipilih
print("\nDaftar senyawa dengan status 'Dipilih':")
for i, senyawa in enumerate(dipilih['Compound'].unique(), start=1):
    print(f"{i}. {senyawa}")

print("\nDaftar senyawa dengan status 'Tidak dipilih':")
for i, senyawa in enumerate(tidak_dipilih['Compound'].unique(), start=1):
    print(f"{i}. {senyawa}")

# --- Preview hasil SwissADME ---
pd.set_option('display.max_rows', 20)
print("\nPreview hasil SwissADME:")
print(df_swiss.head())


Jumlah baris verifikasi_senyawa: 17
Jumlah baris SwissADME: 17

✅ Data SwissADME berhasil disimpan di sheet 'swissadme_senyawa' dalam file D:/semhas/Data/senyawa_pubchem.xlsx

📊 Ringkasan Data Senyawa:
Jumlah senyawa unik   : 16 senyawa
Jumlah total SMILES   : 17 SMILES

Daftar senyawa unik:
1. 4-aminobenzoic acid
2. Citric acid
3. L-phenylalanine
4. Trans-3-indoleacrylic acid
5. DL-tryptophan
6. 4-O-feruloyl-D-quinic acid
7. Murrangatin
8. Alpha-lapachone
9. Hainanmurpanin
10. Murraol
11. Murralongin
12. (1R,9S)-5-[(E)-2-(4-chlorophenyl)ethenyl]-11-(pyrimidine-5-carbonyl)-7,11-diazatricyclo[7.3.1.02,7]trideca-2,4-dien-6-one
13. Paniculatin
14. Limonene, (+/-)-
15. Pheophorbide a
16. SB236057

📌 Ringkasan Status Akhir:
Senyawa dengan status 'Dipilih'     : 13 senyawa (14 SMILES)
Senyawa dengan status 'Tidak dipilih': 3 senyawa (3 SMILES)

Daftar senyawa dengan status 'Dipilih':
1. 4-aminobenzoic acid
2. Citric acid
3. L-phenylalanine
4. Trans-3-indoleacrylic acid
5. DL-tryptophan
6. Mu

# Akuisisi Protein Target

In [1]:
import pandas as pd

# File Excel
target_file = 'D:/semhas/Data/target.xlsx'

# Baca semua sheet
xls = pd.ExcelFile(target_file)
sheet_names = xls.sheet_names[:14]  # Ambil 14 sheet pertama

all_data = []

for sheet in sheet_names:
    try:
        # Baca mulai dari baris kedua sebagai header
        df = pd.read_excel(target_file, sheet_name=sheet, header=1)
        all_data.append(df)
    except Exception as e:
        print(f"⚠️ Gagal membaca sheet {sheet}: {e}")

# Gabungkan semua sheet
if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    print("\n✅ Semua 14 sheet berhasil digabungkan. Preview:")
    
    # Ambil hanya kolom yang diinginkan
    selected_cols = ['Target Name','UniProt ID','Min Activity','Assay type','Probability']
    df_selected = df_all.reindex(columns=selected_cols)
    
    print(df_selected.head(20))
    
    # --- Tambahkan ringkasan jumlah ---
    total_rows = len(df_selected)
    print(f"\nJumlah total baris: {total_rows}")
    print("\nJumlah baris non-NA per kolom:")
    print(df_selected.count())
    
    # Simpan ke sheet baru
    with pd.ExcelWriter(target_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_selected.to_excel(writer, sheet_name='all_14_sheets', index=False)
    
    print(f"\n✅ Data berhasil disimpan di sheet 'all_14_sheets' dalam file {target_file}")
else:
    print("❌ Tidak ada data yang berhasil dibaca.")



✅ Semua 14 sheet berhasil digabungkan. Preview:
                                          Target Name UniProt ID Min Activity  \
0                Thyroid stimulating hormone receptor     P16473     125.9 nm   
1                                     NADPH oxidase 1     Q9Y5S8          NaN   
2                          Ras-related protein Rab-9A     P51151          NaN   
3                               Kruppel-like factor 5     Q13887          NaN   
4                             Niemann-Pick C1 protein     O15118          NaN   
5           Transcription intermediary factor 1-alpha     O15164          NaN   
6                             Proteasome component C5     P20618          NaN   
7   Endoplasmic reticulum-associated amyloid beta-...     Q99714          NaN   
8                                    Cyclooxygenase-1     P23219          NaN   
9                                       Transthyretin     P02766          NaN   
10                       Proteasome Macropain subunit     P4

In [2]:
import pandas as pd

# File Excel
file_path = 'D:/semhas/Data/target.xlsx'

# Baca sheet all_14_sheets
df_all = pd.read_excel(file_path, sheet_name='all_14_sheets')

# Pastikan kolom Probability bertipe numerik
df_all['Probability'] = pd.to_numeric(df_all['Probability'], errors='coerce')

# Filter berdasarkan kondisi
df_filtered = df_all[(df_all['Probability'] >= 0.55) | (df_all['Assay type'] == 'Potency')]

# Hitung jumlah sebelum menghapus duplikat
jumlah_sebelum = len(df_filtered)

# Hapus duplikat hanya berdasarkan UniProt ID
df_filtered_unique = df_filtered.drop_duplicates(subset=['UniProt ID'])
jumlah_setelah = len(df_filtered_unique)

# Simpan ke sheet baru
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_filtered_unique.to_excel(writer, sheet_name='targets_filtered_unique', index=False)

print(f"✅ Data berhasil difilter dan disimpan di sheet 'targets_filtered_unique'.")
print(f"Jumlah baris Probability >= 0,55 dan Assay type = Potency : {jumlah_sebelum}")
print(f"Jumlah baris unik berdasarkan UniProt ID: {jumlah_setelah}")


✅ Data berhasil difilter dan disimpan di sheet 'targets_filtered_unique'.
Jumlah baris Probability >= 0,55 dan Assay type = Potency : 819
Jumlah baris unik berdasarkan UniProt ID: 229


In [65]:
import pandas as pd

# File Excel
file_path = 'D:/semhas/Data/target.xlsx'

# --- Baca sheet targets_filtered_unique ---
df_targets = pd.read_excel(file_path, sheet_name='targets_filtered_unique', engine='openpyxl')

# Nama kolom UniProt ID
col_targets = 'UniProt ID'  # pastikan sesuai nama kolom di sheet

# --- Hapus baris kosong (opsional) ---
df_targets = df_targets.dropna(subset=[col_targets])

# --- Tampilkan isi kolom UniProt ID ---
print(f"📄 Isi kolom '{col_targets}' di sheet 'targets_filtered_unique':")
print(df_targets[col_targets].reset_index(drop=True))

# --- Statistik ---
print(f"\n📊 Total UniProt ID di 'targets_filtered_unique' (setelah hapus NaN): {df_targets[col_targets].count()}")
print(f"📊 Jumlah UniProt ID unik: {df_targets[col_targets].nunique()}")


📄 Isi kolom 'UniProt ID' di sheet 'targets_filtered_unique':
0      P16473
1      Q9Y5S8
2      P51151
3      Q13887
4      O15118
        ...  
224    P30542
225    P55263
226    P48775
227    P43004
228    Q8N4C8
Name: UniProt ID, Length: 229, dtype: object

📊 Total UniProt ID di 'targets_filtered_unique' (setelah hapus NaN): 229
📊 Jumlah UniProt ID unik: 229


In [3]:
import pandas as pd

# File Excel sumber dan tujuan
input_file = 'D:/semhas/Data/target.xlsx'
output_file = 'D:/semhas/Data/target.xlsx'  # bisa sama dengan file sumber

# Sheet yang ingin digabungkan
sheet_names = ['Sheet16', 'Sheet12', 'Sheet14(limone)']

all_data = []

for sheet in sheet_names:
    try:
        df = pd.read_excel(input_file, sheet_name=sheet, engine='openpyxl', header=1)
        df['Sheet'] = sheet  # kolom asal sheet
        all_data.append(df)
        print(f"✅ Sheet {sheet} berhasil dibaca, baris: {len(df)}")
    except Exception as e:
        print(f"⚠️ Gagal membaca sheet {sheet}: {e}")

if all_data:
    # Gabungkan semua sheet
    df_combined = pd.concat(all_data, ignore_index=True)
    print(f"\n✅ Semua sheet berhasil digabungkan. Total baris: {len(df_combined)}")

    # Hitung total UniProt ID (pecah jika ada lebih dari satu ID dalam satu sel)
    if 'Uniprot ID' in df_combined.columns:
        df_uniprot_expanded = df_combined['Uniprot ID'].astype(str).str.split(' ').explode()
        total_uniprot = len(df_uniprot_expanded)
        unique_uniprot = df_uniprot_expanded.drop_duplicates().count()
        duplicate_uniprot = total_uniprot - unique_uniprot

        print("\n📊 Statistik UniProt ID setelah gabungan:")
        print(f"Total UniProt ID (dihitung semua, termasuk duplikat): {total_uniprot}")
        print(f"Jumlah UniProt ID unik: {unique_uniprot}")
        print(f"Jumlah UniProt ID duplikat: {duplicate_uniprot}")

        # Ambil dan tampilkan UniProt ID unik
        df_unique_uniprot = df_uniprot_expanded.drop_duplicates().reset_index(drop=True)
        print("\n✅ Daftar UniProt ID unik:")
        print(df_unique_uniprot)

        # Simpan daftar unik ke sheet baru (opsional)
        with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df_combined.to_excel(writer, sheet_name='swisstarget_gabungan', index=False)
            df_unique_uniprot.to_excel(writer, sheet_name='UniProt_ID_unique', index=False)

        print("\n✅ Data gabungan dan UniProt ID unik berhasil disimpan di file Excel.")
    else:
        print("⚠️ Kolom 'Uniprot ID' tidak ditemukan di sheet gabungan.")
else:
    print("❌ Tidak ada data yang berhasil dibaca.")


✅ Sheet Sheet16 berhasil dibaca, baris: 100
✅ Sheet Sheet12 berhasil dibaca, baris: 100
✅ Sheet Sheet14(limone) berhasil dibaca, baris: 93

✅ Semua sheet berhasil digabungkan. Total baris: 293

📊 Statistik UniProt ID setelah gabungan:
Total UniProt ID (dihitung semua, termasuk duplikat): 305
Jumlah UniProt ID unik: 263
Jumlah UniProt ID duplikat: 42

✅ Daftar UniProt ID unik:
0      P51679
1      P51681
2      Q12809
3      O95136
4      Q8TDU6
        ...  
258    P27361
259    P24723
260    P80365
261    P30304
262    Q06124
Name: Uniprot ID, Length: 263, dtype: object

✅ Data gabungan dan UniProt ID unik berhasil disimpan di file Excel.


In [6]:
import pandas as pd

# File Excel
file_path = 'D:/semhas/Data/target.xlsx'

# Baca sheet gabungan
df = pd.read_excel(file_path, sheet_name='swisstarget_gabungan', engine='openpyxl', header=0)

if 'Uniprot ID' in df.columns and 'Probability*' in df.columns:
    # Filter baris dengan Probability* > 0
    df_filtered = df[df['Probability*'] > 0]

    # Pecah UniProt ID menjadi baris terpisah sambil tetap menjaga kolom lain
    df_expanded = df_filtered.assign(Uniprot_ID_single=df_filtered['Uniprot ID'].astype(str).str.split(' ')).explode('Uniprot_ID_single')

    # Hapus duplikat berdasarkan UniProt ID (tetap simpan baris pertama dari setiap ID)
    df_unique = df_expanded.drop_duplicates(subset='Uniprot_ID_single').reset_index(drop=True)

    # Tampilkan preview
    print("✅ Daftar baris dengan UniProt ID unik dan Probability* > 0:")
    print(df_unique.head(20))

    # Simpan ke sheet baru
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_unique.to_excel(writer, sheet_name='UniProt_ID_unique_filtered', index=False)

    print("\n✅ Data dengan kolom lengkap dan UniProt ID unik berhasil disimpan di sheet 'UniProt_ID_unique_filtered'")

else:
    if 'Uniprot ID' not in df.columns:
        print("⚠️ Kolom 'Uniprot ID' tidak ditemukan di sheet.")
    if 'Probability*' not in df.columns:
        print("⚠️ Kolom 'Probability*' tidak ditemukan di sheet.")


✅ Daftar baris dengan UniProt ID unik dan Probability* > 0:
                                               Target   Common name  \
0                       C-C chemokine receptor type 4          CCR4   
1                       C-C chemokine receptor type 5          CCR5   
2                                                HERG         KCNH2   
3              Sphingosine 1-phosphate receptor Edg-5         S1PR2   
4              G-protein coupled bile acid receptor 1        GPBAR1   
5                                    Integrin alpha-4         ITGA4   
6                 Squalene synthetase   (by homology)         FDFT1   
7                             Integrin alpha-V/beta-3   ITGAV ITGB3   
8                             Integrin alpha-V/beta-3   ITGAV ITGB3   
9                           Integrin alpha-IIb/beta-3  ITGA2B ITGB3   
10                            Integrin alpha-V/beta-5   ITGB5 ITGAV   
11                            Integrin alpha-V/beta-6   ITGAV ITGB6   
12               

In [7]:
import pandas as pd

# File Excel
target_file = 'D:/semhas/Data/target.xlsx'

# --- Baca sheet ---
df_targets = pd.read_excel(target_file, sheet_name='targets_filtered_unique', engine='openpyxl')
df_swiss = pd.read_excel(target_file, sheet_name='UniProt_ID_unique_filtered', engine='openpyxl')

# Nama kolom UniProt ID
col_targets = 'UniProt ID'        # dari sheet targets_filtered_unique
col_swiss = 'Uniprot_ID_single'   # dari sheet UniProt_ID_unique_filtered

# --- Hapus baris kosong (NaN) ---
df_targets = df_targets.dropna(subset=[col_targets])
df_swiss = df_swiss.dropna(subset=[col_swiss])

# --- Ambil hanya kolom UniProt ID masing-masing sheet ---
df_targets_id = df_targets[[col_targets]].rename(columns={col_targets: 'Uniprot ID'})
df_swiss_id = df_swiss[[col_swiss]].rename(columns={col_swiss: 'Uniprot ID'})

# --- Gabungkan keduanya ---
df_combined = pd.concat([df_targets_id, df_swiss_id], ignore_index=True)

# --- Ambil UniProt ID unik ---
df_unique = df_combined.drop_duplicates().reset_index(drop=True)

# --- Tampilkan preview ---
print("✅ UniProt ID unik gabungan (50 baris pertama):")
print(df_unique.head(50))

# --- Statistik ---
print(f"\n📊 Total UniProt ID setelah gabungan (termasuk duplikat): {len(df_combined)}")
print(f"📊 Jumlah UniProt ID unik: {len(df_unique)}")

# --- Simpan ke sheet baru ---
with pd.ExcelWriter(target_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_unique.to_excel(writer, sheet_name='UniProt_ID_merged_unique', index=False)

print("\n✅ Hasil UniProt ID unik gabungan berhasil disimpan di sheet 'UniProt_ID_merged_unique'")


✅ UniProt ID unik gabungan (50 baris pertama):
   Uniprot ID
0      P16473
1      Q9Y5S8
2      P51151
3      Q13887
4      O15118
5      O15164
6      P20618
7      Q99714
8      P23219
9      P02766
10     P49721
11     P28482
12     P34995
13     Q06124
14     Q92753
15     Q9Y345
16     P19838
17     P30926
18     P27695
19     P24557
20     P11166
21     O15151
22     P42224
23     P02708
24     P23415
25     P30305
26     O14672
27     P07384
28     P09211
29     P21462
30     Q9HC97
31     P08684
32     O00257
33     Q16236
34     Q6V1X1
35     P25090
36     Q16288
37     P67870
38     P51955
39     P08173
40     O75469
41     P17706
42     P11387
43     P43116
44     P22736
45     P08912
46     P07477
47     P42680
48     Q00535
49     O60341

📊 Total UniProt ID setelah gabungan (termasuk duplikat): 282
📊 Jumlah UniProt ID unik: 273

✅ Hasil UniProt ID unik gabungan berhasil disimpan di sheet 'UniProt_ID_merged_unique'


In [ ]:
# Jumlah Entrez Gene ID Dari OMIM

In [23]:
import pandas as pd

# 📁 Path ke file Excel
file_path = "D:/semhas/Data/OMIM-Entry.xlsx"

# 🔹 Baca file dari baris ke-5 (karena header berada di baris ke-5)
df = pd.read_excel(file_path, header=4)

# 🔹 Filter baris dengan Entrez Gene ID tidak kosong
df_filtered = df[df["Entrez Gene ID"].notna()].copy()
total_filtered = len(df_filtered)

# 🔹 Cari duplikat berdasarkan Entrez Gene ID
df_duplicates = df_filtered[df_filtered.duplicated(subset=["Entrez Gene ID"], keep=False)].copy()
total_duplikat = len(df_duplicates)

# 🔹 Simpan ke file (Filtered: Sheet2, Duplicates: Sheet3)
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_filtered.to_excel(writer, sheet_name="Filtered", index=False)
    df_duplicates.to_excel(writer, sheet_name="Duplicates", index=False)

# 🔹 Tampilkan ringkasan
print("📊 Ringkasan:")
print(f"🔹 Setelah filter (Entrez Gene ID tidak kosong): {total_filtered}")
print(f"🔹 Jumlah baris duplikat berdasarkan Entrez Gene ID: {total_duplikat}")


📊 Ringkasan:
🔹 Setelah filter (Entrez Gene ID tidak kosong): 176
🔹 Jumlah baris duplikat berdasarkan Entrez Gene ID: 0


In [ ]:
# Cek Validasi UniProt ID dengan Taxonomy ID: 9606(Homo sapiens)  dari target

In [37]:
import pandas as pd

# --- File input ---
target_file = "D:/semhas/Data/target.xlsx"
mapping_file = "D:/semhas/Data/HUMAN_9606_idmapping.dat"

# --- Baca sheet "UniProt_ID_merged_unique" ---
df_target = pd.read_excel(target_file, sheet_name="UniProt_ID_merged_unique")

# Normalisasi nama kolom
df_target.columns = df_target.columns.str.strip()

# Pastikan nama kolom UniProt sesuai
col_uniprot = "Uniprot ID"

# --- Baca file mapping UniProt ---
df_map = pd.read_csv(mapping_file, sep="\t", header=None,
                     names=["UniProtKB-AC", "ID_Type", "Value"])

# --- Ambil mapping UniProt → Gene Name ---
df_gene = df_map[df_map["ID_Type"] == "Gene_Name"][["UniProtKB-AC", "Value"]]
df_gene.rename(columns={"UniProtKB-AC": "UniProt", "Value": "Gene Name"}, inplace=True)

# --- Merge dengan gene name ---
df_merge = df_target.merge(df_gene, how="left", left_on=col_uniprot, right_on="UniProt")

# --- Tambahkan Status Verifikasi ---
valid_uniprot_set = set(df_map["UniProtKB-AC"].unique())
df_merge["Status Verifikasi"] = df_merge[col_uniprot].apply(
    lambda x: "✅ Valid (Homo sapiens)" if pd.notna(x) and x in valid_uniprot_set else "❌ Tidak valid"
)

# --- Tambahkan kolom Sumber ---
df_merge["Sumber"] = "SuperPred"

# --- Format final ---
df_final = pd.DataFrame({
    "No.": range(1, len(df_merge) + 1),
    "ID Asal": df_merge[col_uniprot],
    "UniProt": df_merge[col_uniprot],
    "Status Verifikasi": df_merge["Status Verifikasi"],
    "Gene Name": df_merge["Gene Name"],
    "Sumber": df_merge["Sumber"]
})

# --- Simpan ke sheet baru ---
with pd.ExcelWriter(target_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_final.to_excel(writer, sheet_name="SuperPred_UniProt_validated", index=False)

print("✅ Hasil validasi sudah disimpan ke sheet 'SuperPred_UniProt_validated'")
print(df_final.head(10))


✅ Hasil validasi sudah disimpan ke sheet 'SuperPred_UniProt_validated'
   No. ID Asal UniProt       Status Verifikasi Gene Name     Sumber
0    1  P16473  P16473  ✅ Valid (Homo sapiens)      TSHR  SuperPred
1    2  Q9Y5S8  Q9Y5S8  ✅ Valid (Homo sapiens)      NOX1  SuperPred
2    3  P51151  P51151  ✅ Valid (Homo sapiens)     RAB9A  SuperPred
3    4  Q13887  Q13887  ✅ Valid (Homo sapiens)      KLF5  SuperPred
4    5  O15118  O15118  ✅ Valid (Homo sapiens)      NPC1  SuperPred
5    6  O15164  O15164  ✅ Valid (Homo sapiens)    TRIM24  SuperPred
6    7  P20618  P20618  ✅ Valid (Homo sapiens)     PSMB1  SuperPred
7    8  Q99714  Q99714  ✅ Valid (Homo sapiens)  HSD17B10  SuperPred
8    9  P23219  P23219  ✅ Valid (Homo sapiens)     PTGS1  SuperPred
9   10  P02766  P02766  ✅ Valid (Homo sapiens)       TTR  SuperPred


In [ ]:
# Cek Validasi UniProt ID dengan Taxonomy ID: 9606(Homo sapiens)  dari omim

In [35]:
import pandas as pd

# --- File input ---
target_file = "D:/semhas/Data/OMIM-Entry.xlsx"
mapping_file = "D:/semhas/Data/HUMAN_9606_idmapping.dat"

# --- Baca sheet "Filtered" ---
df_target = pd.read_excel(target_file, sheet_name="Filtered")

# Normalisasi nama kolom
df_target.columns = df_target.columns.str.strip()

# Tentukan nama kolom ID Asal
col_id_asal = "Entrez Gene ID"

# Pastikan kolom Entrez di target jadi string
df_target[col_id_asal] = df_target[col_id_asal].astype(str)

# --- Baca file mapping UniProt ---
df_map = pd.read_csv(mapping_file, sep="\t", header=None,
                     names=["UniProtKB-AC", "ID_Type", "Value"])

# --- Ambil mapping Entrez Gene ID → UniProt ---
df_entrez2uniprot = df_map[df_map["ID_Type"] == "GeneID"][["Value", "UniProtKB-AC"]]
df_entrez2uniprot.rename(columns={"Value": col_id_asal, "UniProtKB-AC": "UniProt"}, inplace=True)

# Pastikan kolom Entrez di mapping juga string
df_entrez2uniprot[col_id_asal] = df_entrez2uniprot[col_id_asal].astype(str)

# --- Ambil mapping UniProt → Gene Name ---
df_gene = df_map[df_map["ID_Type"] == "Gene_Name"][["UniProtKB-AC", "Value"]]
df_gene.rename(columns={"UniProtKB-AC": "UniProt", "Value": "Gene Name"}, inplace=True)

# --- Merge Entrez dengan UniProt ---
df_merge = df_target.merge(df_entrez2uniprot, how="left", on=col_id_asal)

# --- Tambahkan Gene Name ---
df_merge = df_merge.merge(df_gene, how="left", on="UniProt")

# --- Tambahkan Status Verifikasi ---
valid_uniprot_set = set(df_map["UniProtKB-AC"].unique())
df_merge["Status Verifikasi"] = df_merge["UniProt"].apply(
    lambda x: "✅ Valid (Homo sapiens)" if pd.notna(x) and x in valid_uniprot_set else "❌ Tidak valid"
)

# --- Tambahkan kolom Sumber ---
df_merge["Sumber"] = "OMIM"

# --- Format final ---
df_final = pd.DataFrame({
    "No.": range(1, len(df_merge) + 1),
    "ID Asal": df_merge[col_id_asal],
    "UniProt": df_merge["UniProt"],
    "Status Verifikasi": df_merge["Status Verifikasi"],
    "Gene Name": df_merge["Gene Name"],
    "Sumber": df_merge["Sumber"]
})

# --- Simpan ke sheet baru ---
with pd.ExcelWriter(target_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_final.to_excel(writer, sheet_name="OMIM_UniProt_validated", index=False)

print("✅ Hasil validasi sudah disimpan ke sheet 'OMIM_UniProt_validated'")
print(df_final.head(10))


✅ Hasil validasi sudah disimpan ke sheet 'OMIM_UniProt_validated'
   No. ID Asal UniProt       Status Verifikasi Gene Name Sumber
0    1    7350  P25874  ✅ Valid (Homo sapiens)      UCP1   OMIM
1    2    7350  Q4KMT7  ✅ Valid (Homo sapiens)      UCP1   OMIM
2    3   26027  Q8WXI4  ✅ Valid (Homo sapiens)    ACOT11   OMIM
3    4   54976  Q9GZN8  ✅ Valid (Homo sapiens)    ADISSP   OMIM
4    5  388581  Q5T7M4  ✅ Valid (Homo sapiens)  C1QTNF12   OMIM
5    6   56246  Q8TCY5  ✅ Valid (Homo sapiens)      MRAP   OMIM
6    7   51129  Q9BY76  ✅ Valid (Homo sapiens)   ANGPTL4   OMIM
7    8   10891  Q9UBK2  ✅ Valid (Homo sapiens)  PPARGC1A   OMIM
8    9    8841  O15379  ✅ Valid (Homo sapiens)     HDAC3   OMIM
9   10    5743  P35354  ✅ Valid (Homo sapiens)     PTGS2   OMIM


In [ ]:
# Mengabungkan omim dan superpred

In [38]:
import pandas as pd

# --- File input ---
omim_file = "D:/semhas/Data/OMIM-Entry.xlsx"
superpred_file = "D:/semhas/Data/target.xlsx"
output_file = "D:/semhas/Data/All_UniProt_validated.xlsx"  # bisa jadi file gabungan baru

# --- Baca masing-masing sheet hasil validasi ---
df_omim = pd.read_excel(omim_file, sheet_name="OMIM_UniProt_validated")
df_superpred = pd.read_excel(superpred_file, sheet_name="SuperPred_UniProt_validated")

# --- Gabungkan keduanya ---
df_combined = pd.concat([df_superpred, df_omim], ignore_index=True)

# --- Reset nomor urut ---
df_combined["No."] = range(1, len(df_combined) + 1)

# --- Simpan ke file baru ---
df_combined.to_excel(output_file, sheet_name="All_UniProt_validated", index=False)

print("✅ Gabungan OMIM + SuperPred sudah disimpan ke file:", output_file)
print(df_combined.head(10))


✅ Gabungan OMIM + SuperPred sudah disimpan ke file: D:/semhas/Data/All_UniProt_validated.xlsx
   No. ID Asal UniProt       Status Verifikasi Gene Name     Sumber
0    1  P16473  P16473  ✅ Valid (Homo sapiens)      TSHR  SuperPred
1    2  Q9Y5S8  Q9Y5S8  ✅ Valid (Homo sapiens)      NOX1  SuperPred
2    3  P51151  P51151  ✅ Valid (Homo sapiens)     RAB9A  SuperPred
3    4  Q13887  Q13887  ✅ Valid (Homo sapiens)      KLF5  SuperPred
4    5  O15118  O15118  ✅ Valid (Homo sapiens)      NPC1  SuperPred
5    6  O15164  O15164  ✅ Valid (Homo sapiens)    TRIM24  SuperPred
6    7  P20618  P20618  ✅ Valid (Homo sapiens)     PSMB1  SuperPred
7    8  Q99714  Q99714  ✅ Valid (Homo sapiens)  HSD17B10  SuperPred
8    9  P23219  P23219  ✅ Valid (Homo sapiens)     PTGS1  SuperPred
9   10  P02766  P02766  ✅ Valid (Homo sapiens)       TTR  SuperPred


In [ ]:
#cek duplikat

In [41]:
import pandas as pd

# --- File input/output ---
file_path = "D:/semhas/Data/All_UniProt_validated.xlsx"

# --- Baca sheet gabungan ---
df_all = pd.read_excel(file_path, sheet_name="All_UniProt_validated")

# --- Pastikan kolom yang dipakai ada ---
print("Kolom tersedia:", df_all.columns.tolist())

# --- Cari duplikat berdasarkan kombinasi ID Asal + UniProt ---
dupes = df_all[df_all.duplicated(subset=["ID Asal", "UniProt"], keep=False)]

# --- Data unik (hapus duplikat) ---
df_unique = df_all.drop_duplicates(subset=["ID Asal", "UniProt"]).reset_index(drop=True)
df_unique["No."] = range(1, len(df_unique) + 1)

# --- Statistik ---
total_data = len(df_all)
total_dupes = len(dupes)
total_unique = len(df_unique)

print("\n📊 Statistik Data All_UniProt_validated")
print(f"Total data gabungan : {total_data}")
print(f"Jumlah duplikat     : {total_dupes}")
print(f"Jumlah unik         : {total_unique}")

# --- Buat DataFrame ringkasan statistik ---
df_stats = pd.DataFrame({
    "Keterangan": ["Total Data", "Jumlah Duplikat", "Jumlah Unik"],
    "Jumlah": [total_data, total_dupes, total_unique]
})

# --- Simpan ke file Excel ---
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    dupes.to_excel(writer, sheet_name="Duplicated", index=False)
    df_unique.to_excel(writer, sheet_name="Unique_UniProt", index=False)
    df_stats.to_excel(writer, sheet_name="Statistik", index=False)

print(f"\n✅ Hasil duplikat, unik, dan statistik sudah disimpan di file {file_path}")


Kolom tersedia: ['No.', 'ID Asal', 'UniProt', 'Status Verifikasi', 'Gene Name', 'Sumber']

📊 Statistik Data All_UniProt_validated
Total data gabungan : 578
Jumlah duplikat     : 0
Jumlah unik         : 578

✅ Hasil duplikat, unik, dan statistik sudah disimpan di file D:/semhas/Data/All_UniProt_validated.xlsx


In [50]:
import pandas as pd
import os

# 📁 Folder sumber & file output
folder_path = "D:/semhas/Data/hasil_Superpred"
output_file = "D:/semhas/Data/Superpred_Dipilih.xlsx"

# Kolom penting
selected_columns = ["Target Name", "UniProt ID", "Min Activity", "Assay type", 
                    "Probability", "Model accuracy"]

dataframes = []
excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]

for file in excel_files:
    file_path = os.path.join(folder_path, file)
    print(f"📂 Membaca file: {file_path}")

    try:
        xls = pd.ExcelFile(file_path, engine="openpyxl")
        for sheet_name in xls.sheet_names:
            print(f"   🔹 Sheet: {sheet_name}")
            df = pd.read_excel(xls, sheet_name=sheet_name, header=1, engine="openpyxl")
            df.columns = df.columns.str.strip()

            available_columns = [col for col in selected_columns if col in df.columns]
            if not available_columns:
                continue

            df_filtered = df[available_columns].copy()
            df_filtered["Source_File"] = file
            df_filtered["Source_Sheet"] = sheet_name
            dataframes.append(df_filtered)

    except Exception as e:
        print(f"❌ Gagal membaca {file_path}: {e}")

# Gabungkan dan proses data
if dataframes:
    merged_df = pd.concat(dataframes, ignore_index=True)
    merged_df = merged_df.dropna(subset=["UniProt ID"])

    # Data terpilih berdasarkan dua kondisi:
    # 1. Probability >= 0.5 (jika kolom tersedia)
    # 2. ATAU Assay type == "Potency" dan Min Activity tidak kosong
    cond_probability = (merged_df.get("Probability") >= 0.5) if "Probability" in merged_df.columns else False
    cond_activity = (merged_df["Assay type"].fillna("") == "Potency") & (merged_df["Min Activity"].notna())

    df_dipilih = merged_df[(cond_probability | cond_activity)].copy()

    # Hilangkan duplikat berdasarkan UniProt ID
    df_dipilih = df_dipilih.drop_duplicates(subset=["UniProt ID"], keep="first").reset_index(drop=True)

    # Ringkasan
    unique_proteins = merged_df["UniProt ID"].nunique()
    summary_df = pd.DataFrame({
        "Keterangan": ["Jumlah Total Entri", "Jumlah Protein Target Unik", "Jumlah Terpilih"],
        "Jumlah": [len(merged_df), unique_proteins, len(df_dipilih)]
    })

    # Simpan ke Excel
    with pd.ExcelWriter(output_file, engine="openpyxl", mode="w") as writer:
        merged_df.to_excel(writer, index=False, sheet_name="Data Gabungan")
        df_dipilih.to_excel(writer, index=False, sheet_name="Dipilih")
        summary_df.to_excel(writer, index=False, sheet_name="Ringkasan")

    print(f"✅ Selesai. Data disimpan di: {output_file}")
else:
    print("❌ Tidak ada file Excel valid ditemukan.")


📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets1 - 2025-07-19T054204.825.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets1 - 2025-07-19T054214.688.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets10 - 2025-07-19T055450.251.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets11 - 2025-07-19T055542.195.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets13 - 2025-07-19T055930.578.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets2 - 2025-07-19T054402.358.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets3 - 2025-07-19T054634.502.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets4 - 2025-07-19T054832.108.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets5 - 2025-07-19T054941.371.xlsx
   🔹 Sheet: Sheet1
📂 Membaca file: D:/semhas/Data/hasil_Superpred\Targets5 - 202

In [12]:
print("Kolom yang tersedia:", df.columns.tolist())


Kolom yang tersedia: ['MIM Number', 'Title', 'Included Titles', 'Cytogenetic Location', 'Genomic Coordinates (From NCBI/GRCh38)', 'Entrez Gene ID', 'UniProt', 'Gene_Name', 'Status Verifikasi']


In [ ]:
# Cek Validasi UniProt ID dengan Taxonomy ID: 9606(Homo sapiens) dari superpred

In [14]:
import pandas as pd

# File Excel dengan UniProt ID unik
target_file = "D:/semhas/Data/target.xlsx"
sheet_name = "UniProt_ID_merged_unique"

# File mapping UniProt -> Gene Name
mapping_file = "D:/semhas/Data/HUMAN_9606_idmapping.dat"

# --- Baca sheet UniProt ID unik ---
df_uniprot = pd.read_excel(target_file, sheet_name=sheet_name, engine='openpyxl')

# --- Baca file mapping UniProt ---
# File .dat biasanya tab-delimited: UniProtKB-AC  ID-type  ID
df_mapping = pd.read_csv(mapping_file, sep='\t', header=None, names=['UniProt ID', 'ID Type', 'ID'])

# --- Filter mapping hanya untuk Gene Name ---
df_gene = df_mapping[df_mapping['ID Type'] == 'Gene_Name'][['UniProt ID', 'ID']]
df_gene = df_gene.rename(columns={'ID': 'Gene Name'})

# --- Merge dengan UniProt ID unik ---
df_merged = pd.merge(df_uniprot, df_gene, on='Uniprot ID', how='left')

# --- Tambahkan kolom Status Verifikasi ---
df_merged['Status Verifikasi'] = '✅ Valid (Homo sapiens)'

# --- Preview ---
print("✅ Preview hasil dengan Gene Name:")
print(df_merged.head(20))

# --- Simpan ke sheet baru ---
with pd.ExcelWriter(target_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_merged.to_excel(writer, sheet_name='UniProt_with_GeneName', index=False)

print("\n✅ Hasil berhasil disimpan di sheet 'UniProt_with_GeneName'")


KeyError: 'UniProt ID'

In [9]:
import pandas as pd

# === 1. File sumber ===
superpred_file = "D:/semhas/Data/SuperPred_Dipilih.xlsx"
mapping_file = "D:/semhas/Data/HUMAN_9606_idmapping.dat"

# === 2. Baca data SuperPred ===
df_superpred = pd.read_excel(superpred_file, sheet_name="Dipilih")
df_superpred = df_superpred.rename(columns=lambda x: x.strip())  # hapus spasi tersembunyi

# === 3. Baca data mapping
mapping_df = pd.read_csv(mapping_file, sep='\t', header=None, names=["UniProt", "DB", "ID"])

# === 4. Ambil semua UniProt valid
valid_uniprot = mapping_df["UniProt"].unique()

# === 5. Tambahkan kolom status verifikasi
df_superpred['Status Verifikasi'] = df_superpred['UniProt ID'].isin(valid_uniprot).map(
    lambda x: '✅ Valid (Homo sapiens)' if x else '❌ Tidak valid / Non-Human'
)

# === 6. Tambahkan kolom Gene Name
gene_map = mapping_df[mapping_df["DB"] == "Gene_Name"].copy()
gene_map = gene_map.rename(columns={"UniProt": "UniProt ID", "ID": "Gene Name"}).drop(columns=["DB"])

# Gabungkan dengan DataFrame SuperPred berdasarkan UniProt ID
df_superpred = pd.merge(df_superpred, gene_map, on="UniProt ID", how="left")

# === 7. Simpan ke sheet baru
with pd.ExcelWriter(superpred_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_superpred.to_excel(writer, sheet_name='Verifikasi-UniProt', index=False)

# === 8. Rekap jumlah hasil verifikasi
rekap = df_superpred['Status Verifikasi'].value_counts()
print("\n📊 Rekapitulasi Status Verifikasi:")
print(rekap)

# === 9. Contoh hasil
print("\n📌 Contoh hasil dengan Gene Name:")
print(df_superpred[['UniProt ID', 'Gene Name', 'Status Verifikasi']].head())



📊 Rekapitulasi Status Verifikasi:
Status Verifikasi
✅ Valid (Homo sapiens)    269
Name: count, dtype: int64

📌 Contoh hasil dengan Gene Name:
  UniProt ID Gene Name       Status Verifikasi
0     P16473      TSHR  ✅ Valid (Homo sapiens)
1     Q9Y5S8      NOX1  ✅ Valid (Homo sapiens)
2     P51151     RAB9A  ✅ Valid (Homo sapiens)
3     Q13887      KLF5  ✅ Valid (Homo sapiens)
4     O15118      NPC1  ✅ Valid (Homo sapiens)


In [ ]:
# Gabungkan OMIM dan Superpred

In [11]:
# === Sumber: Proses penggabungan data protein target dari OMIM dan SuperPred ===
# Tujuan: Membuat satu tabel gabungan yang berisi ID asal, UniProt ID, status verifikasi, nama gen, dan sumber data

import pandas as pd

# === 1. File sumber ===
omim_file = "D:/semhas/Data/OMIM-Entry.xlsx"                # Sumber data OMIM (Filtered)
superpred_file = "D:/semhas/Data/SuperPred_Dipilih.xlsx"    # Sumber data hasil verifikasi SuperPred

# === 2. Baca sheet 'Filtered' dari OMIM ===
df_omim = pd.read_excel(omim_file, sheet_name="Filtered")
df_omim = df_omim.rename(columns=lambda x: x.strip())  # Hilangkan spasi tersembunyi pada nama kolom

# Ambil kolom yang dibutuhkan & ganti nama kolom agar konsisten
df_omim = df_omim[['Entrez Gene ID', 'UniProt', 'Gene_Name', 'Status Verifikasi']].copy()
df_omim = df_omim.rename(columns={
    'Entrez Gene ID': 'ID Asal',
    'UniProt': 'UniProt',
    'Gene_Name': 'Gene Name'
})
df_omim['ID Asal'] = df_omim['ID Asal'].astype(str)  # Pastikan ID Asal berupa string
df_omim['Sumber'] = 'OMIM'

# === 3. Baca sheet 'Verifikasi-UniProt' dari SuperPred ===
df_superpred = pd.read_excel(superpred_file, sheet_name="Verifikasi-UniProt")
df_superpred = df_superpred.rename(columns=lambda x: x.strip())  # Hilangkan spasi tersembunyi

# Ambil kolom yang dibutuhkan & samakan nama kolom
df_superpred = df_superpred[['UniProt ID', 'Status Verifikasi', 'Gene Name']].copy()
df_superpred = df_superpred.rename(columns={'UniProt ID': 'UniProt'})
df_superpred['ID Asal'] = df_superpred['UniProt']
df_superpred['Sumber'] = 'SuperPred'

# Susun ulang kolom agar sama
df_superpred = df_superpred[['ID Asal', 'UniProt', 'Status Verifikasi', 'Gene Name', 'Sumber']]
df_omim = df_omim[['ID Asal', 'UniProt', 'Status Verifikasi', 'Gene Name', 'Sumber']]

# === 4. Gabungkan data dari OMIM dan SuperPred ===
df_all = pd.concat([df_omim, df_superpred], ignore_index=True)

# Tambahkan kolom nomor urut di paling kiri
df_all.insert(0, 'No.', range(1, len(df_all) + 1))

# === 5. Tampilkan hasil ===
print(df_all.head(20))  # Tampilkan 20 baris pertama
print(f"\n✅ Total gabungan entri: {len(df_all)}")

# (Opsional) Simpan ke file Excel jika diinginkan:
df_all.to_excel("D:/semhas/Data/Target_Protein.xlsx", index=False)


    No. ID Asal     UniProt       Status Verifikasi Gene Name Sumber
0     1    7350      P25874  ✅ Valid (Homo sapiens)      UCP1   OMIM
1     2    7350      Q4KMT7  ✅ Valid (Homo sapiens)      UCP1   OMIM
2     3   26027      Q8WXI4  ✅ Valid (Homo sapiens)    ACOT11   OMIM
3     4   54976      Q9GZN8  ✅ Valid (Homo sapiens)    ADISSP   OMIM
4     5  388581      Q5T7M4  ✅ Valid (Homo sapiens)  C1QTNF12   OMIM
5     6   56246      Q8TCY5  ✅ Valid (Homo sapiens)      MRAP   OMIM
6     7   51129      Q9BY76  ✅ Valid (Homo sapiens)   ANGPTL4   OMIM
7     8   10891      Q9UBK2  ✅ Valid (Homo sapiens)  PPARGC1A   OMIM
8     9    8841      O15379  ✅ Valid (Homo sapiens)     HDAC3   OMIM
9    10    5743      P35354  ✅ Valid (Homo sapiens)     PTGS2   OMIM
10   11     657      P36894  ✅ Valid (Homo sapiens)    BMPR1A   OMIM
11   12    9572      P20393  ✅ Valid (Homo sapiens)     NR1D1   OMIM
12   13    9572      F1D8S3  ✅ Valid (Homo sapiens)     NR1D1   OMIM
13   14  133522      Q86YN6  ✅ Val

In [12]:
import pandas as pd

# === 1. Baca file Excel ===
input_file = "D:/semhas/Data/Target_Protein.xlsx"

# Baca sheet pertama (diasumsikan gabungan data)
xls = pd.ExcelFile(input_file)
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
df.columns = df.columns.str.strip()

# Tambahkan kolom sumber berdasarkan nilai ID Asal (OMIM: angka, SuperPred: string UniProt)
df['Sumber'] = df['ID Asal'].apply(lambda x: 'OMIM' if str(x).isdigit() else 'SuperPred')

# === 2. Filter yang valid (case-insensitive) ===
df['Status Verifikasi'] = df['Status Verifikasi'].str.strip()
df_valid = df[df['Status Verifikasi'].str.lower() == '✅ Valid (Homo sapiens)'].copy()

# === 3. Simpan ke sheet baru 'Dipilih' ===
with pd.ExcelWriter(input_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_valid.to_excel(writer, sheet_name='Dipilih', index=False)

# === 4. Hitung jumlah valid/tidak valid per sumber ===
rekap = df.groupby(['Sumber', 'Status Verifikasi']).size().unstack(fill_value=0)

# === 6. Tampilkan hasil ===
print("📊 Rekapitulasi Status Verifikasi per Sumber:")
print(rekap)


# Hitung total valid
valid_df = df_all[df_all['Status Verifikasi'].str.contains("Valid", na=False)]

# Total valid semua sumber
total_valid = len(valid_df)

# Cetak hasil
print(f"\n🔢 Total Data Valid dari Semua Sumber: {total_valid}")



📊 Rekapitulasi Status Verifikasi per Sumber:
Status Verifikasi  ✅ Valid (Homo sapiens)  ❌ Tidak ditemukan
Sumber                                                      
OMIM                                  301                  4
SuperPred                             269                  0

🔢 Total Data Valid dari Semua Sumber: 570


In [13]:
import pandas as pd

# === 1. Baca file Excel ===
input_file = "D:/semhas/Data/Target_Protein.xlsx"

# Baca sheet pertama (diasumsikan gabungan data)
xls = pd.ExcelFile(input_file)
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
df.columns = df.columns.str.strip()

# === 2. Tambahkan kolom 'Sumber' berdasarkan ID Asal (OMIM: angka, SuperPred: UniProt)
df['Sumber'] = df['ID Asal'].apply(lambda x: 'OMIM' if str(x).isdigit() else 'SuperPred')

# === 3. Bersihkan kolom Status Verifikasi
df['Status Verifikasi'] = df['Status Verifikasi'].str.strip()

# === 4. Filter data valid
df_valid = df[df['Status Verifikasi'].str.lower().str.contains("valid")].copy()

# === 5. Simpan hasil valid ke sheet baru "Dipilih"
with pd.ExcelWriter(input_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_valid.to_excel(writer, sheet_name='Dipilih', index=False)
print("✅ Data valid berhasil disimpan ke sheet 'Dipilih'.")

# === 6. Rekap status verifikasi berdasarkan sumber
rekap = df.groupby(['Sumber', 'Status Verifikasi']).size().unstack(fill_value=0)

# === 7. Hitung total valid dari semua sumber
total_valid = len(df_valid)

# === 8. Tampilkan hasil
print("\n📊 Rekapitulasi Status Verifikasi per Sumber:")
print(rekap)
print(f"\n🔢 Total Data Valid dari Semua Sumber: {total_valid}")


✅ Data valid berhasil disimpan ke sheet 'Dipilih'.

📊 Rekapitulasi Status Verifikasi per Sumber:
Status Verifikasi  ✅ Valid (Homo sapiens)  ❌ Tidak ditemukan
Sumber                                                      
OMIM                                  301                  4
SuperPred                             269                  0

🔢 Total Data Valid dari Semua Sumber: 570


In [ ]:
# Hapus duplikat Gabungan

In [14]:
import pandas as pd

# Baca file dan sheet
file_path = 'D:/semhas/Data/Target_Protein.xlsx'
sheet_asal = 'Dipilih'
sheet_tujuan = 'Tanpa_Duplikat'

# Baca data
df = pd.read_excel(file_path, sheet_name=sheet_asal)

# Tampilkan kolom yang tersedia
print("Kolom tersedia:", df.columns)

# Nama kolom UniProt
kolom_uniprot = 'UniProt'

# Cek apakah kolom tersedia
if kolom_uniprot in df.columns:
    # Cek dan tampilkan duplikat
    duplikat = df[df.duplicated(subset=[kolom_uniprot], keep=False)]
    print(f"\nJumlah duplikat ditemukan: {duplikat.shape[0]}")

    # Hapus duplikat dan simpan hasil
    df_tanpa_duplikat = df.drop_duplicates(subset=[kolom_uniprot], keep='first')

    # Simpan ke sheet baru di file yang sama
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_tanpa_duplikat.to_excel(writer, sheet_name=sheet_tujuan, index=False)

    print(f"Hasil tanpa duplikat disimpan ke sheet '{sheet_tujuan}'.")
else:
    print(f"Kolom '{kolom_uniprot}' tidak ditemukan.")


Kolom tersedia: Index(['No.', 'ID Asal', 'UniProt', 'Status Verifikasi', 'Gene Name',
       'Sumber'],
      dtype='object')

Jumlah duplikat ditemukan: 18
Hasil tanpa duplikat disimpan ke sheet 'Tanpa_Duplikat'.


In [16]:
import pandas as pd

# Baca file dan sheet
file_path = 'D:/semhas/Data/Target_Protein.xlsx'
sheet_asal = 'Dipilih'
sheet_tujuan = 'Tanpa_Duplikat'

# Baca data
df = pd.read_excel(file_path, sheet_name=sheet_asal)

# Tampilkan kolom yang tersedia
print("Kolom tersedia:", df.columns)

# Nama kolom UniProt
kolom_uniprot = 'UniProt'

# Cek apakah kolom tersedia
if kolom_uniprot in df.columns:
    print(f"\nJumlah total entri awal: {len(df)}")
    print(f"Contoh data awal:\n{df[[kolom_uniprot]].head(10)}")

    # Cek dan tampilkan duplikat
    duplikat = df[df.duplicated(subset=[kolom_uniprot], keep=False)]
    print(f"\nJumlah duplikat ditemukan: {len(duplikat)}")
    if not duplikat.empty:
        print(f"Contoh duplikat:\n{duplikat[[kolom_uniprot]].drop_duplicates().head(10)}")

    # Hapus duplikat dan simpan hasil
    df_tanpa_duplikat = df.drop_duplicates(subset=[kolom_uniprot], keep='first')
    print(f"\nJumlah entri setelah penghapusan duplikat: {len(df_tanpa_duplikat)}")

    # Simpan ke sheet baru di file yang sama
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_tanpa_duplikat.to_excel(writer, sheet_name=sheet_tujuan, index=False)

    print(f"Hasil tanpa duplikat disimpan ke sheet '{sheet_tujuan}'.")
else:
    print(f"Kolom '{kolom_uniprot}' tidak ditemukan.")


Kolom tersedia: Index(['No.', 'ID Asal', 'UniProt', 'Status Verifikasi', 'Gene Name',
       'Sumber'],
      dtype='object')

Jumlah total entri awal: 570
Contoh data awal:
  UniProt
0  P25874
1  Q4KMT7
2  Q8WXI4
3  Q9GZN8
4  Q5T7M4
5  Q8TCY5
6  Q9BY76
7  Q9UBK2
8  O15379
9  P35354

Jumlah duplikat ditemukan: 18
Contoh duplikat:
    UniProt
8    O15379
9    P35354
48   P08235
83   P10275
172  Q8TDU6
240  P10646
247  P05186
248  P00750
279  Q13370

Jumlah entri setelah penghapusan duplikat: 561
Hasil tanpa duplikat disimpan ke sheet 'Tanpa_Duplikat'.


# Data PPI

In [111]:
import pandas as pd
import requests
import time

# Path ke file Excel yang berisi UniProt ID
file_path = "D:/semhas/projek/Target_Protein.xlsx"

# Baca kolom UniProt_ID
df = pd.read_excel(file_path, sheet_name='Dipilih')
uniprot_ids = df["UniProt"].dropna().unique().tolist()

# Fungsi untuk memproses batch dan ambil data PPI dari STRING
def fetch_ppi_batch(batch_ids, batch_number, total_batches):
    print(f"🚀 Memproses batch {batch_number} dari {total_batches} ({len(batch_ids)} ID)...")

    # Format string untuk query
    protein_query = "%0d%0a".join(batch_ids)

    # URL API STRING, dengan parameter 'species=9606' untuk Homo sapiens dan 'required_score=400' untuk skor minimal 0,4
    url = f"https://string-db.org/api/tsv/network?identifiers={protein_query}&species=9606&required_score=400"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            batch_file = f"ppi_batch_{batch_number}.tsv"
            with open(batch_file, "w", encoding="utf-8") as f:
                f.write(response.text)
            print(f"✅ Batch {batch_number} selesai! Data disimpan di {batch_file}")
            return pd.read_csv(batch_file, sep="\t")
        else:
            print(f"❌ Error Batch {batch_number}: {response.status_code} - {response.text}")
            return pd.DataFrame()
    except Exception as e:
        print(f"❌ Gagal mengunduh data pada batch {batch_number}: {e}")
        return pd.DataFrame()

# 🔄 Looping batch
batch_size = 100
all_results = []
total_batches = (len(uniprot_ids) + batch_size - 1) // batch_size

for i in range(0, len(uniprot_ids), batch_size):
    batch_ids = uniprot_ids[i:i+batch_size]
    batch_number = (i // batch_size) + 1
    df_batch = fetch_ppi_batch(batch_ids, batch_number, total_batches)
    if not df_batch.empty:
        all_results.append(df_batch)
    time.sleep(1.5)  # Hindari overload server STRING

# 🔗 Gabungkan semua hasil
if all_results:
    combined_df = pd.concat(all_results, ignore_index=True)
    output_file = "D:/semhas/projek/PPI.tsv"
    combined_df.to_csv(output_file, sep="\t", index=False)
    print(f"\n🎉 Semua batch selesai! Data PPI gabungan disimpan di:\n{output_file}")
else:
    print("⚠️ Tidak ada data PPI yang berhasil diambil.")


🚀 Memproses batch 1 dari 6 (100 ID)...
✅ Batch 1 selesai! Data disimpan di ppi_batch_1.tsv
🚀 Memproses batch 2 dari 6 (100 ID)...
✅ Batch 2 selesai! Data disimpan di ppi_batch_2.tsv
🚀 Memproses batch 3 dari 6 (100 ID)...
✅ Batch 3 selesai! Data disimpan di ppi_batch_3.tsv
🚀 Memproses batch 4 dari 6 (100 ID)...
✅ Batch 4 selesai! Data disimpan di ppi_batch_4.tsv
🚀 Memproses batch 5 dari 6 (100 ID)...
✅ Batch 5 selesai! Data disimpan di ppi_batch_5.tsv
🚀 Memproses batch 6 dari 6 (61 ID)...
✅ Batch 6 selesai! Data disimpan di ppi_batch_6.tsv

🎉 Semua batch selesai! Data PPI gabungan disimpan di:
D:/semhas/projek/PPI.tsv


In [113]:
import pandas as pd
import networkx as nx

# File PPI dan mapping
ppi_file = 'D:/semhas/projek/PPI.tsv'
mapping_file = 'D:/semhas/projek/9606.protein.aliases.v12.0.txt'

# Baca data PPI
df = pd.read_csv(ppi_file, sep='\t')

# Baca mapping STRING ID ke UniProt
mapping_df = pd.read_csv(mapping_file, sep='\t', header=None, names=['string_id', 'alias', 'source'])
uniprot_map = mapping_df[mapping_df['source'].str.contains('UniProt', case=False, na=False)]
string_to_uniprot = dict(zip(uniprot_map['string_id'], uniprot_map['alias']))

# Mapping UniProt ID berdasarkan stringId_A dan stringId_B
df['UniProt_ID_1'] = df['stringId_A'].map(string_to_uniprot)
df['UniProt_ID_2'] = df['stringId_B'].map(string_to_uniprot)

# Hapus baris yang tidak berhasil dimapping
df_clean = df.dropna(subset=['UniProt_ID_1', 'UniProt_ID_2'])

# Buat graph dari pasangan UniProt ID
G = nx.from_pandas_edgelist(df_clean, 'UniProt_ID_1', 'UniProt_ID_2')

# Hitung node dan edge
jumlah_node = G.number_of_nodes()
jumlah_interaksi = G.number_of_edges()

print(f"Jumlah protein (node) yang berinteraksi: {jumlah_node}")
print(f"Jumlah interaksi antar protein (edge): {jumlah_interaksi}")
print(df_clean[['stringId_A', 'stringId_B', 'UniProt_ID_1', 'UniProt_ID_2']].head())


Jumlah protein (node) yang berinteraksi: 332
Jumlah interaksi antar protein (edge): 933
             stringId_A            stringId_B UniProt_ID_1 UniProt_ID_2
0  9606.ENSP00000216180  9606.ENSP00000471477   uc003bei.1   uc002pkn.2
1  9606.ENSP00000216180  9606.ENSP00000287820   uc003bei.1   uc003bwr.4
2  9606.ENSP00000216180  9606.ENSP00000312652   uc003bei.1   uc003vml.3
3  9606.ENSP00000216180  9606.ENSP00000348069   uc003bei.1   uc002grt.3
4  9606.ENSP00000216180  9606.ENSP00000244289   uc003bei.1   uc002otr.4


In [18]:
import pandas as pd

# 📂 File PPI dan mapping
ppi_file = 'D:/semhas/projek/hasil/PPI_Combined_Final.tsv'
mapping_file = 'D:/semhas/projek/hasil/9606.protein.aliases.v12.0.txt'

# 📥 Baca file PPI
df_ppi = pd.read_csv(ppi_file, sep='\t')

# Rename kolom 'score' menjadi 'combined_score' jika perlu
df_ppi = df_ppi.rename(columns={'score': 'combined_score'})

# Filter hanya interaksi dengan combined_score ≥ 0.4
df_filtered = df_ppi[df_ppi['combined_score'] >= 0.4].copy()

# 📥 Baca mapping STRING ID → UniProt ID
df_map = pd.read_csv(mapping_file, sep='\t', header=None, names=['string_id', 'alias', 'source'])
df_map_uniprot = df_map[df_map['source'].str.contains('UniProt', case=False, na=False)]

# Buat dictionary untuk mapping
string_to_uniprot = dict(zip(df_map_uniprot['string_id'], df_map_uniprot['alias']))

# 🧭 Mapping ke UniProt ID
df_filtered['UniProt_ID_1'] = df_filtered['stringId_A'].map(string_to_uniprot)
df_filtered['UniProt_ID_2'] = df_filtered['stringId_B'].map(string_to_uniprot)

# Hapus baris yang tidak berhasil dimapping
df_final = df_filtered.dropna(subset=['UniProt_ID_1', 'UniProt_ID_2'])

# 🎯 Tabel akhir yang ingin ditampilkan
result = df_final[['stringId_A', 'stringId_B', 'UniProt_ID_1', 'UniProt_ID_2', 'combined_score']]
print(result.head(10))

# 📊 Statistik
jumlah_interaksi = len(result)
protein_unik = pd.unique(result[['UniProt_ID_1', 'UniProt_ID_2']].values.ravel())
jumlah_protein_unik = len(protein_unik)

print("\n📊 Ringkasan Interaksi PPI:")
print(f"🔹 Jumlah total interaksi (edge)         : {jumlah_interaksi}")
print(f"🔸 Jumlah protein unik yang berinteraksi : {jumlah_protein_unik}")

# 📝 Parameter ekstraksi dari STRING
print("\n⚙️ Parameter ekstraksi STRING:")
print("- Organisme: Homo sapiens (TaxID: 9606)")
print("- Jenis interaksi: full STRING network (eksperimental, prediktif, dan teks)")
print("- Skor kepercayaan minimum (combined_score): 0.4 (medium confidence)")

# 💾 Simpan hasil
result.to_csv('D:/semhas/projek/hasil/PPI_Table_Filtered_0.4.tsv', sep='\t', index=False)
print("\n✅ Tabel PPI dengan mapping UniProt disimpan.")


             stringId_A            stringId_B UniProt_ID_1 UniProt_ID_2  \
0  9606.ENSP00000215832  9606.ENSP00000381607   uc010gtk.2       V9HWE9   
1  9606.ENSP00000215832  9606.ENSP00000411532   uc010gtk.2   uc002huq.4   
2  9606.ENSP00000215832  9606.ENSP00000380252   uc010gtk.2   uc002ulg.6   
3  9606.ENSP00000215832  9606.ENSP00000351908   uc010gtk.2   uc003qhc.4   
4  9606.ENSP00000215832  9606.ENSP00000216714   uc010gtk.2   uc058yte.1   
5  9606.ENSP00000215832  9606.ENSP00000321656   uc010gtk.2   uc003lcp.3   
6  9606.ENSP00000215832  9606.ENSP00000501150   uc010gtk.2   uc011bmq.4   
7  9606.ENSP00000215832  9606.ENSP00000355966   uc010gtk.2   uc001hir.3   
8  9606.ENSP00000215832  9606.ENSP00000428056   uc010gtk.2   uc003jva.4   
9  9606.ENSP00000215832  9606.ENSP00000354558   uc010gtk.2   uc001asd.4   

   combined_score  
0           0.411  
1           0.421  
2           0.430  
3           0.495  
4           0.513  
5           0.563  
6           0.618  
7           0.

# Data ddi

In [ ]:
#UniProt --> Domain PFam

In [14]:
import requests
import pandas as pd
import time
from tqdm import tqdm

# === 1. BACA UNIPROT ID DARI FILE ===
file_path = 'D:/semhas/projek/Target_Protein.xlsx'
df = pd.read_excel(file_path, sheet_name='Tanpa_Duplikat')

all_uniprot_ids = df['UniProt'].dropna().unique().tolist()

# === 2. FUNGSI AMBIL DOMAIN DARI INTERPRO (Pfam) ===
def get_pfam_domains(uniprot_id):
    url = f"https://www.ebi.ac.uk/interpro/api/entry/pfam/protein/uniprot/{uniprot_id}"
    headers = {"Accept": "application/json"}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            domains = [item['metadata']['accession'] for item in data['results']]
            return domains
        else:
            return []
    except Exception as e:
        print(f"❌ Error untuk {uniprot_id}: {e}")
        return []

# === 3. LOOPING MAPPING DENGAN OUTPUT DETAIL ===
results = []

print(f"🔍 Memulai pemetaan {len(all_uniprot_ids)} UniProt ID ke domain Pfam...\n")
for idx, uniprot_id in enumerate(tqdm(all_uniprot_ids, desc="Memproses"), 1):
    domains = get_pfam_domains(uniprot_id)
    
    if domains:
        for domain in domains:
            results.append({'UniProt_ID': uniprot_id, 'Pfam_ID': domain})
        domain_str = ', '.join(domains)
        print(f"[{idx}] {uniprot_id} → {len(domains)} domain: {domain_str}")
    else:
        print(f"[{idx}] {uniprot_id} → Tidak ditemukan domain Pfam.")
    
    time.sleep(1)  # Hindari rate limit

# === 4. SIMPAN HASIL KE FILE EXCEL ===
df_result = pd.DataFrame(results)
output_file = 'D:/semhas/projek/TargetProtein_PFam.xlsx'
df_result.to_excel(output_file, index=False)

print(f"\n✅ Pemetaan selesai! Hasil disimpan di: {output_file}")

# === 5. SIMPAN SEMUA UNIPROT ID KE FILE EXCEL ===
df_all_ids = pd.DataFrame({'UniProt_ID': all_uniprot_ids})
all_uniprot_file = 'D:/semhas/projek/All_UniProt_IDs.xlsx'
df_all_ids.to_excel(all_uniprot_file, index=False)

print(f"✅ Daftar semua UniProt ID disimpan di: {all_uniprot_file}")


🔍 Memulai pemetaan 561 UniProt ID ke domain Pfam...



Memproses:   0%|                                                                               | 0/561 [00:00<?, ?it/s]

[1] P25874 → 1 domain: PF00153


Memproses:   0%|▏                                                                      | 1/561 [00:02<27:53,  2.99s/it]

[2] Q4KMT7 → 1 domain: PF00153


...

[560] P00746 → 1 domain: PF00089


Memproses: 100%|████████████████████████████████████████████████████████████████████▉| 560/561 [21:05<00:02,  2.11s/it]

[561] P29375 → 7 domain: PF00628, PF01388, PF02373, PF02375, PF02928, PF08429, PF21323


Memproses: 100%|█████████████████████████████████████████████████████████████████████| 561/561 [21:07<00:00,  2.26s/it]



✅ Pemetaan selesai! Hasil disimpan di: D:/semhas/projek/TargetProtein_PFam.xlsx
✅ Daftar semua UniProt ID disimpan di: D:/semhas/projek/All_UniProt_IDs.xlsx


In [6]:
# periksa duplikat UniPOrtId --> PfamId

In [22]:
import pandas as pd

# 1. Baca file Excel
file_path = 'D:/semhas/projek/TargetProtein_PFam.xlsx'
df = pd.read_excel(file_path)

# 2. Tampilkan nama kolom untuk memastikan struktur
print("Kolom tersedia:", df.columns.tolist())

# Misalnya kolom bernama 'UniProt_ID' dan 'Pfam_ID' (ubah sesuai nama aktual kolom)
uniprot_col = 'UniProt_ID'
pfam_col = 'Pfam_ID'

# 3. Simpan kolom Pfam_ID asli ke kolom baru
df['Pfam_ID_Original'] = df[pfam_col]

# 4. Bersihkan kolom Pfam_ID → Hapus versi (contoh: PF00069.23 → PF00069)
df[pfam_col] = df[pfam_col].astype(str).str.extract(r'^(PF\d{5})')

# 5. Pindahkan 'Pfam_ID_Original' sebelum 'Pfam_ID'
cols = df.columns.tolist()
pfam_idx = cols.index(pfam_col)
# Hapus kolom dulu agar bisa disisipkan ulang
cols.remove('Pfam_ID_Original')
# Sisipkan di posisi sebelum 'Pfam_ID'
cols.insert(pfam_idx, 'Pfam_ID_Original')
df = df[cols]

# 6. Tampilkan jumlah total baris
print("Jumlah total relasi (termasuk duplikat):", len(df))

# 7. Cek jumlah relasi unik
unique_pairs = df[[uniprot_col, pfam_col]].drop_duplicates()
print("Jumlah relasi unik UniProt–Pfam:", len(unique_pairs))

# 8. Tampilkan duplikat (jika ingin dicek lebih lanjut)
duplicates = df[df.duplicated(subset=[uniprot_col, pfam_col], keep=False)]
print("Jumlah relasi duplikat:", len(duplicates))
print("\nContoh relasi duplikat:")
print(duplicates.head())

# 9. (Opsional) Simpan relasi unik ke file baru
unique_output = df.drop_duplicates(subset=[uniprot_col, pfam_col])
unique_output.to_excel('D:/semhas/projek/TargetProtein_PFam_Unique.xlsx', index=False)


Kolom tersedia: ['UniProt_ID', 'Pfam_ID']
Jumlah total relasi (termasuk duplikat): 1056
Jumlah relasi unik UniProt–Pfam: 1056
Jumlah relasi duplikat: 0

Contoh relasi duplikat:
Empty DataFrame
Columns: [UniProt_ID, Pfam_ID_Original, Pfam_ID]
Index: []


In [ ]:
# Mapping PFAM --> DDI

In [5]:
import pandas as pd
import re

# === 1. BACA DAN BERSIHKAN DOMAIN ===
file_domain = "D:/semhas/projek/TargetProtein_PFam.xlsx"
df_pfam = pd.read_excel(file_domain)

# Ambil dan bersihkan ID domain (PFxxxxx)
df_pfam['Pfam_ID_Clean'] = df_pfam['Pfam_ID'].astype(str).str.extract(r'(PF\d{5})')
domain_unik = set(df_pfam['Pfam_ID_Clean'].dropna().unique())

print(f"Jumlah domain unik dari protein target: {len(domain_unik)}")

# === 2. PARSE FILE 3DID ===
file_3did = "D:/semhas/projek/3did_flat_Mar_3_2025.dat"
ddi_all = []

with open(file_3did, "r") as f:
    for line in f:
        if line.startswith("#=ID"):
            match = re.findall(r'PF\d{5}', line)
            if len(match) == 2:
                d1, d2 = match
                ddi_all.append((d1, d2))

print(f"Jumlah total pasangan DDI di 3DID: {len(ddi_all)}")

# === 3. COCOKKAN DENGAN DOMAIN UNIK ===
ddi_relevan = [(d1, d2) for d1, d2 in ddi_all if d1 in domain_unik and d2 in domain_unik]

print(f"Jumlah pasangan DDI yang relevan: {len(ddi_relevan)}")

# === 4. SIMPAN KE EXCEL ===
df_ddi_relevan = pd.DataFrame(ddi_relevan, columns=["Domain_1", "Domain_2"])
output_path = "D:/semhas/projek/DDI_Relevan.xlsx"
df_ddi_relevan.to_excel(output_path, index=False)

print(f"✅ DDI relevan disimpan di: {output_path}")


Jumlah domain unik dari protein target: 478
Jumlah total pasangan DDI di 3DID: 20644
Jumlah pasangan DDI yang relevan: 827
✅ DDI relevan disimpan di: D:/semhas/projek/DDI_Relevan.xlsx


In [4]:
import pandas as pd
import re

# === 1. BACA DAN BERSIHKAN DOMAIN ===
file_domain = "D:/semhas/projek/TargetProtein_PFam.xlsx"
df_pfam = pd.read_excel(file_domain)

# Ambil dan bersihkan ID domain (PFxxxxx)
df_pfam['Pfam_ID_Clean'] = df_pfam['Pfam_ID'].astype(str).str.extract(r'(PF\d{5})')
domain_unik = df_pfam['Pfam_ID_Clean'].dropna().unique()

print(f"Jumlah domain unik dari protein target: {len(domain_unik)}")

# === Simpan domain unik ke Excel ===
df_domain_unik = pd.DataFrame(domain_unik, columns=['Domain_Unik'])
output_domain = "D:/semhas/projek/Domain_Unik_Protein.xlsx"
df_domain_unik.to_excel(output_domain, index=False)
print(f"✅ Domain unik disimpan di: {output_domain}")

# === 2. PARSE FILE 3DID ===
file_3did = "D:/semhas/projek/3did_flat_Mar_3_2025.dat"
ddi_all = []

with open(file_3did, "r") as f:
    for line in f:
        if line.startswith("#=ID"):
            match = re.findall(r'PF\d{5}', line)
            if len(match) == 2:
                d1, d2 = match
                ddi_all.append((d1, d2))

print(f"Jumlah total pasangan DDI di 3DID: {len(ddi_all)}")

# === 3. COCOKKAN DENGAN DOMAIN UNIK ===
set_domain_unik = set(domain_unik)
ddi_relevan = [(d1, d2) for d1, d2 in ddi_all if d1 in set_domain_unik and d2 in set_domain_unik]

print(f"Jumlah pasangan DDI yang relevan: {len(ddi_relevan)}")

# === 4. SIMPAN DDI RELEVAN KE EXCEL ===
df_ddi_relevan = pd.DataFrame(ddi_relevan, columns=["Domain_1", "Domain_2"])
output_ddi = "D:/semhas/projek/DDI_Relevan.xlsx"
df_ddi_relevan.to_excel(output_ddi, index=False)

print(f"✅ DDI relevan disimpan di: {output_ddi}")


Jumlah domain unik dari protein target: 478
✅ Domain unik disimpan di: D:/semhas/projek/Domain_Unik_Protein.xlsx
Jumlah total pasangan DDI di 3DID: 20644
Jumlah pasangan DDI yang relevan: 827
✅ DDI relevan disimpan di: D:/semhas/projek/DDI_Relevan.xlsx


In [ ]:
# Menentukan PDA

In [19]:
import pandas as pd
from itertools import product

# 1. Load data
ppi_df = pd.read_csv("D:/semhas/projek/string_interactions_short.tsv", sep="\t")
pfam_df = pd.read_excel("D:/semhas/projek/TargetProtein_PFam.xlsx")
ddi_df = pd.read_excel("D:/semhas/projek/DDI_Pasangan_Relevan.xlsx")
mapping_df = pd.read_csv("D:/semhas/projek/9606.protein.aliases.v12.0.txt", sep='\t', header=None, names=["string_id", "alias", "source"])

# 2. Filter mapping dari UniProt
mapping_df = mapping_df[mapping_df['source'] == 'Ensembl_UniProt']
enspid_to_uniprot = dict(zip(mapping_df["string_id"], mapping_df["alias"]))

# 3. Mapping UniProt ke PPI
ppi_df["Protein_1"] = ppi_df["node1_string_id"].map(enspid_to_uniprot)
ppi_df["Protein_2"] = ppi_df["node2_string_id"].map(enspid_to_uniprot)
ppi_df = ppi_df.dropna(subset=["Protein_1", "Protein_2"])

# 4. Preprocessing domain dan DDI
pfam_df['Pfam_ID'] = pfam_df['Pfam_ID'].str.split('.').str[0]
ddi_df['Domain_1'] = ddi_df['Domain_1'].str.split('.').str[0]
ddi_df['Domain_2'] = ddi_df['Domain_2'].str.split('.').str[0]
ddi_pairs = set(tuple(sorted([a, b])) for a, b in zip(ddi_df['Domain_1'], ddi_df['Domain_2']))

# 5. Mapping protein ke daftar domain
pfam_dict = pfam_df.groupby("UniProt_ID")["Pfam_ID"].apply(list).to_dict()

# 6. Gabungkan semua PPI + cek interaksi domain
rows = []
for _, row in ppi_df.iterrows():
    p1, p2 = row["Protein_1"], row["Protein_2"]
    domains1 = pfam_dict.get(p1, [])
    domains2 = pfam_dict.get(p2, [])
    
    # Cek apakah minimal satu pasangan domain saling berinteraksi
    interaksi_ditemukan = any(tuple(sorted([d1, d2])) in ddi_pairs for d1, d2 in product(domains1, domains2))
    
    rows.append({
        "Protein_1": p1,
        "Protein_2": p2,
        "Domains_Protein_1": ", ".join(domains1) if domains1 else "-",
        "Domains_Protein_2": ", ".join(domains2) if domains2 else "-",
        "Interaksi_Domain": "Yes" if interaksi_ditemukan else "No"
    })

# 7. Simpan hasil
hasil_df = pd.DataFrame(rows)
output_file = "D:/semhas/projek/PPI_DDI_Merged.xlsx"
hasil_df.to_excel(output_file, index=False)

print(f"Data gabungan disimpan di: {output_file}")
print("Jumlah total pasangan PPI:", len(hasil_df))
print("Jumlah pasangan dengan interaksi domain:", (hasil_df["Interaksi_Domain"] == "Yes").sum())


Data gabungan disimpan di: D:/semhas/projek/PPI_DDI_Merged.xlsx
Jumlah total pasangan PPI: 3880
Jumlah pasangan dengan interaksi domain: 340


In [18]:
import pandas as pd
from itertools import product

# 1. Load data
ppi_df = pd.read_csv("D:/semhas/projek/string_interactions_short.tsv", sep="\t")
pfam_df = pd.read_excel("D:/semhas/projek/TargetProtein_PFam.xlsx")
ddi_df = pd.read_excel("D:/semhas/projek/DDI_Pasangan_Relevan.xlsx")
mapping_df = pd.read_csv("D:/semhas/projek/9606.protein.aliases.v12.0.txt", sep='\t', header=None, names=["string_id", "alias", "source"])

# 2. Filter mapping hanya dari UniProt
mapping_df = mapping_df[mapping_df['source'] == 'Ensembl_UniProt']

# 3. Buat mapping ENSP → UniProt
enspid_to_uniprot = dict(zip(mapping_df["string_id"], mapping_df["alias"]))

# 4. Tambahkan UniProt ID ke PPI
ppi_df["UniProt_1"] = ppi_df["node1_string_id"].map(enspid_to_uniprot)
ppi_df["UniProt_2"] = ppi_df["node2_string_id"].map(enspid_to_uniprot)

# 5. Preprocessing domain
ddi_df['Domain_1'] = ddi_df['Domain_1'].str.split('.').str[0]
ddi_df['Domain_2'] = ddi_df['Domain_2'].str.split('.').str[0]
pfam_df['Pfam_ID'] = pfam_df['Pfam_ID'].str.split('.').str[0]

# 6. Buat set DDI
ddi_pairs = set(tuple(sorted([a, b])) for a, b in zip(ddi_df['Domain_1'], ddi_df['Domain_2']))

# 7. Buat dict protein → domain
pfam_dict = pfam_df.groupby("UniProt_ID")["Pfam_ID"].apply(list).to_dict()

# 8. Cek PDA
pda_rows = []
for _, row in ppi_df.iterrows():
    p1, p2 = row["UniProt_1"], row["UniProt_2"]
    domains1 = pfam_dict.get(p1, [])
    domains2 = pfam_dict.get(p2, [])
    for d1, d2 in product(domains1, domains2):
        if tuple(sorted([d1, d2])) in ddi_pairs:
            pda_rows.append({
                "Protein_1": p1,
                "Protein_2": p2,
                "Domain_1": d1,
                "Domain_2": d2
            })
            break

# 9. Hasil
pda_df = pd.DataFrame(pda_rows)

# Hitung jumlah pasangan protein unik
jumlah_pasangan_protein = len(pda_df[['Protein_1', 'Protein_2']].drop_duplicates())

# Hitung jumlah pasangan domain unik
jumlah_pasangan_domain = len(pda_df[['Domain_1', 'Domain_2']].drop_duplicates())

print("Jumlah pasangan PDA yang valid (protein-domain):", len(pda_df))
print("Jumlah pasangan protein–protein unik:", jumlah_pasangan_protein)
print("Jumlah pasangan domain–domain unik:", jumlah_pasangan_domain)
print(pda_df.head())

# 10. Simpan ke file Excel
output_path = "D:/semhas/projek/Hasil_PDA.xlsx"
pda_df.to_excel(output_path, index=False)
print(f"Hasil PDA disimpan di: {output_path}")



Jumlah pasangan PDA yang valid (protein-domain): 340
Jumlah pasangan protein–protein unik: 340
Jumlah pasangan domain–domain unik: 79
  Protein_1 Protein_2 Domain_1 Domain_2
0    Q2M2I8    P08172  PF00069  PF00001
1    Q9UKU7    P28330  PF00441  PF00441
2    Q8WXI4    Q9NPJ3  PF03061  PF03061
3    P36896    P36894  PF00069  PF00069
4    P36896    P18075  PF01064  PF00019
Hasil PDA disimpan di: D:/semhas/projek/Hasil_PDA.xlsx
